In [1]:
import requests
import re
import json
import pandas as pd
import time
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os

In [2]:
# 1. Obtener el HTML de la página del ranking
load_dotenv()
url = os.getenv("TARGET_URL_RESERVES")
BASE_URL = os.getenv("BASE_URL")

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()  # Lanza error si la petición falla

In [3]:
# 2. Parsear HTML con BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
# 3. Buscar la tabla principal y extraer los links
table = soup.find("table")
country_links = []


if table:
    rows = table.find_all("tr")
    for row in rows:
        cell = row.find("td")
        if cell and cell.a:
            relative_link = cell.a["href"]
            full_url = f"{BASE_URL}{relative_link}"
            country_links.append(full_url)

# Convertir la lista en DataFrame
df_links = pd.DataFrame(country_links, columns=["url"])

# Guardar como archivo Parquet
df_links.to_parquet("data/country_reserves_links.parquet", index=False)

In [5]:
# 4. Mostrar resultado
print("Links encontrados:", df_links.size)

Links encontrados: 162


In [6]:
# configuracion necesaria para descargar datos

master_df = pd.DataFrame()

pattern = r'arrayToDataTable\((\[\[.*?\]\])\)'

sleep_duration = 5

In [7]:
# Cargar los países válidos desde el archivo parquet
df_countries = pd.read_parquet("data/countries_with_hpi.parquet")
valid_countries = set(df_countries["country"].unique())

In [8]:
start_time = time.time()
for url in country_links:
    # Extraer país desde el URL
    country = urlparse(url).path.strip("/").split("/")[0]

    # Verificar si el país está en la lista válida
    if country not in valid_countries:
        print(f"⏭️ País {country} no está en la lista, se omite.")
        continue

    try:
        response = requests.get(url, headers=headers)
        html = response.text

        match = re.search(pattern, html, re.DOTALL)
        if match:
            data_str = match.group(1)
            data = json.loads(data_str)

            df = pd.DataFrame(data[1:], columns=data[0])
            df.columns = ["Period", "Value"]
            df["Country"] = country

            df_pivot = df.pivot(index="Period", columns="Country", values="Value")
            master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

            print(f"✅ Datos extraídos correctamente para {country}")
        else:
            print(f"⚠️ No se encontraron datos en {url}")
    except Exception as e:
        print(f"❌ Error procesando {url}: {e}")
    
    time.sleep(sleep_duration)

end_time = time.time()
total_sleep_time = len(valid_countries) * sleep_duration
effective_time = end_time - start_time - total_sleep_time

print(f"⏳ Tiempo total (incluyendo esperas): {end_time - start_time:.2f} s")
print(f"😴 Tiempo en espera: {total_sleep_time:.2f} s")
print(f"⚡ Tiempo efectivo de ejecución: {effective_time:.2f} s")

⏭️ País Albania no está en la lista, se omite.
⏭️ País Algeria no está en la lista, se omite.
⏭️ País Angola no está en la lista, se omite.
⏭️ País Argentina no está en la lista, se omite.
⏭️ País Armenia no está en la lista, se omite.
⏭️ País Aruba no está en la lista, se omite.
✅ Datos extraídos correctamente para Australia
✅ Datos extraídos correctamente para Austria
⏭️ País Azerbaijan no está en la lista, se omite.
⏭️ País Bahamas no está en la lista, se omite.
⏭️ País Bahrain no está en la lista, se omite.
⏭️ País Bangladesh no está en la lista, se omite.
⏭️ País Barbados no está en la lista, se omite.
⏭️ País Belarus no está en la lista, se omite.
✅ Datos extraídos correctamente para Belgium
⏭️ País Belize no está en la lista, se omite.
⏭️ País Benin no está en la lista, se omite.
⏭️ País Bhutan no está en la lista, se omite.
⏭️ País Bolivia no está en la lista, se omite.
⏭️ País Bosnia-and-Herzegovina no está en la lista, se omite.
⏭️ País Botswana no está en la lista, se omite.

In [9]:
master_df.tail()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,Slovenia,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom
Period,,,,,,,,,,,,,,,,,,,,,
Sep 2020,61.56,24.39,27.52,356.61,60.78,90.16,37.82,3281.21,56.99,18.31,...,1.03,916.40,420.5,68.36,517.48,938.00,7949.35,79.68,138.96,208.20
Sep 2021,82.14,28.58,35.26,368.89,63.49,106.27,53.31,3373.02,58.73,24.35,...,1.85,865.06,464.0,78.68,546.08,1007.56,8293.60,121.26,250.08,229.94
Sep 2022,83.58,32.28,39.68,327.58,72.27,102.69,37.78,3193.58,56.34,26.26,...,2.21,1058.84,416.8,90.25,681.21,872.75,7561.94,107.04,230.15,192.22
Sep 2023,90.41,28.41,36.48,340.32,72.84,112.45,41.40,3308.47,57.60,2.85,...,2.20,1152.50,414.1,92.67,634.48,747.86,7736.51,122.20,240.99,203.29
Sep 2024,93.97,32.78,38.21,372.02,81.70,128.07,45.63,3571.80,63.61,2.93,...,2.57,1093.26,420.0,98.56,663.60,802.29,7887.67,152.07,250.62,226.85


In [10]:
# obtain paraguay url
template_url = country_links[0]
country = "Paraguay"

# Separar por "/"
parts = template_url.strip("/").split("/")

# Reemplazar el país (posición 3: después del dominio)

parts[3] = country

# Reconstruir la URL
custom_url = "/".join(parts) + "/"

In [12]:
# extract and include paraguay

response = requests.get(custom_url, headers=headers)
html = response.text

match = re.search(pattern, html, re.DOTALL)
if match:
    data_str = match.group(1)
    data = json.loads(data_str)

    df = pd.DataFrame(data[1:], columns=data[0])
    df.columns = ["Period", "Value"]
    df["Country"] = country

    df_pivot = df.pivot(index="Period", columns="Country", values="Value")
    master_df = pd.merge(master_df, df_pivot, how="outer", left_index=True, right_index=True)

    print(f"✅ Datos extraídos correctamente para {country}")

✅ Datos extraídos correctamente para Paraguay


In [13]:
master_df.tail()

,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,Colombia,Croatia,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
Period,,,,,,,,,,,,,,,,,,,,,
Sep 2020,61.56,24.39,27.52,356.61,60.78,90.16,37.82,3281.21,56.99,18.31,...,916.40,420.5,68.36,517.48,938.00,7949.35,79.68,138.96,208.20,8.95
Sep 2021,82.14,28.58,35.26,368.89,63.49,106.27,53.31,3373.02,58.73,24.35,...,865.06,464.0,78.68,546.08,1007.56,8293.60,121.26,250.08,229.94,9.75
Sep 2022,83.58,32.28,39.68,327.58,72.27,102.69,37.78,3193.58,56.34,26.26,...,1058.84,416.8,90.25,681.21,872.75,7561.94,107.04,230.15,192.22,9.44
Sep 2023,90.41,28.41,36.48,340.32,72.84,112.45,41.40,3308.47,57.60,2.85,...,1152.50,414.1,92.67,634.48,747.86,7736.51,122.20,240.99,203.29,9.99
Sep 2024,93.97,32.78,38.21,372.02,81.70,128.07,45.63,3571.80,63.61,2.93,...,1093.26,420.0,98.56,663.60,802.29,7887.67,152.07,250.62,226.85,9.88


In [14]:
# Resetear el índice para separar "Period"
master_df = master_df.reset_index()

# Mapeo de mes a trimestre
month_to_quarter = {
    "Jan": "Q1", "Feb": "Q1", "Mar": "Q1",
    "Apr": "Q2", "May": "Q2", "Jun": "Q2",
    "Jul": "Q3", "Aug": "Q3", "Sep": "Q3",
    "Oct": "Q4", "Nov": "Q4", "Dec": "Q4"
}

# Extraer mes y año desde "Period"
master_df[["Month", "Year"]] = master_df["Period"].str.extract(r"(\w{3})\s+(\d{4})")

# Filtrar filas válidas (donde se extrajo correctamente el año)
master_df = master_df[master_df["Year"].notna()].copy()

# Mapear trimestre y convertir año
master_df["Quarter"] = master_df["Month"].map(month_to_quarter)
master_df["Year"] = master_df["Year"].astype(int)

# Reordenar las columnas: primero Quarter y Year
cols = ["Quarter", "Year"] + [col for col in master_df.columns if col not in ["Period", "Month", "Quarter", "Year"]]
master_df = master_df[cols]

# Ordenar por Year y Quarter
master_df = master_df.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

In [15]:
master_df.tail()

,Quarter,Year,Australia,Austria,Belgium,Brazil,Bulgaria,Canada,Chile,China,...,South-Africa,South-Korea,Spain,Sweden,Switzerland,Thailand,Turkey,USA,United-Kingdom,Paraguay
782,Q1,2025,105.25,38.00,43.00,336.16,77.14,125.99,45.07,3533.89,...,1238.77,409.7,104.03,672.94,830.08,8322.09,156.51,245.63,226.55,10.37
783,Q2,2025,102.42,37.76,42.53,340.79,75.28,124.73,45.64,3591.33,...,1254.71,404.7,102.70,675.85,NaN,8559.15,138.14,253.84,236.00,10.32
784,Q2,2025,105.64,37.98,42.56,341.46,79.49,125.04,45.78,3593.46,...,1218.34,404.6,102.71,677.81,NaN,8410.57,153.19,254.12,241.53,10.22
785,Q3,2025,NaN,NaN,NaN,NaN,NaN,126.57,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
786,NaN,2025,101.95,36.76,41.58,344.44,79.71,127.86,46.40,3627.08,...,1216.18,410.2,99.83,662.08,NaN,8509.39,149.90,255.68,249.56,10.32


In [16]:
# Guardar como archivo Parquet
master_df.to_parquet("data/reserves_values.parquet", index=False)